<a href="https://colab.research.google.com/github/riveracross/economy_open/blob/main/houseaccount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
print(gspread.__version__)


6.0.2


# 2024_expenditureコード

In [1]:
###2024_expenditureコード###
###Ver.01###
###2024-08-30###

from warnings import resetwarnings
from re import I

# 認証のためのコード
#参考：https://qiita.com/FrozenVoice/items/d9cc04b3f63f4b2d49c2
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

houseaccount ='[【ここにGoogleDriveに保存されているスプレッドシート（Houseaccount_Sheet）のURLを挿入する】]'

#worksheetの2024_houseaccountを選ぶ　番号では(0)となる
worksheet = gc.open_by_url(houseaccount).get_worksheet(0)

import pandas as pd

#[1:]で２行目からをデータ、[0]で１行目をindexに指定している。
#参考：https://note.com/kohaku935/n/nc13bcd11632d
df = pd.DataFrame(worksheet.get_all_values()[1:], columns=worksheet.get_all_values()[0])
df["Date"] = pd.to_datetime(df["Date"])

#DateのSeriesから年月のSeriesを作成 object型に変化している！
df = df.rename({"Date":"YearMonth"},axis="columns")
Date_series =df["YearMonth"].dt.strftime("%Y年%m月")
#print(YearMonth_series)
#LeftItemのSeriesを作成
LItem_series = df["LeftItem"]
#print(LItem_series)
#AmountのSeriesを作成
df["Amount"] = df["Amount"].str.replace(',','')
Amount_series = df["Amount"].astype(int)
#print(Amount_series)
#RightItemのSeriesを作成
RItem_series = df["RightItem"]
#RemarkのSeriesを作成
Remark_series = df["Remark"]
#各Seriesを連結させてDataFrameを作成
df_alldata = pd.concat([LItem_series,Date_series,Amount_series,RItem_series,Remark_series],axis=1)
#print(df)

#df_alldataを使って、df_expenditureデータの作成
df = df_alldata

#不要な行を削除
##LeftItemのうちで不要な科目のリストを除外
##参考：https://nomuotorinko.com/python_isin/
##参考：https://monaledge.com/article/347
#左方の除外リスト
exclude_left = ["Hcash","Ycash","Osav","Rsav","Nsav","Fsav","Nyub","Hyub","Point","Stock","Svalu","Build","Land","Vehi","Rcard","Acard","Mcard","Rassets","Rearning","Vdif","MS","YM","SI","DI","CG","BN","OI"]
#~isinを使って除外する
df = df[~df["LeftItem"].isin(exclude_left)]

#大科目"Unit"に分類
##控除
Deduction = ("cded","ftax","ctax")
df.loc[df["LeftItem"].isin(Deduction),"Unit"] = "1.Deduction"
##固定費
FixedExpenses = ("rent","tel","prov","nhk","sch","edu","yins","nins","hins","water","gas","elec","kero")
df.loc[df["LeftItem"].isin(FixedExpenses),"Unit"] = "2.FixedExpenses"
##変動費
VariableExpenses = ["food","resta","health","dneeds","gasoline","vment","child","culture","social","cloth","study","job","hmade","garden","care","petty"]
df.loc[df["LeftItem"].isin(VariableExpenses),"Unit"] = "3.VariableExpenses"
##特別支出
ExtraordinaryExpenses = ["scost","icost","closs","ecost"]
df.loc[df["LeftItem"].isin(ExtraordinaryExpenses),"Unit"] = "4.ExtraordinaryExpenses"
#print(df_Extra)


#print(df)

#ColumnsのLeftItemをindexに指定
#df = df.set_index("LeftItem")

#データフレームのcolumnsの型を確認
#df.info()

#pivot_tableを使ってインデックスのLeftItemごとの年月日の合計を出す
df_pivot = df.pivot_table(index=["Unit","LeftItem"], columns="YearMonth", values=["Amount"],aggfunc=sum)
#df_pivot = df.pivot_table(index="Date",columns="LeftItem", values=["Amount"],aggfunc=sum)
df_pivot = df_pivot["Amount"].fillna(0)
df_pivot = pd.DataFrame(df_pivot).astype(int)
##fillna(0)をすると小数点以下がつくので、整数(int)に変換
df2 = df_pivot

#インデックスのUnitでソート
df2 = df2.sort_index(level='Unit')

#２つのインデックス、"LeftItem"と"Unit"を列に変換
df2.reset_index(inplace= True)
df2 = df2.rename(columns={'index': 'Items'})

#print(df2)

#LeftItemごとの平均値を求める
df2["Average"] = df2.iloc[:,2:].mean(axis=1).astype(int)

# Averageの列を3列目に移動
target_col = "Average"
df_target = df2[target_col]
##入れ替え対称の列を削除
df2 = df2.drop(target_col, axis=1)
##任意の場所に対象の列を挿入
df2.insert(2, target_col, df_target)
#Unit列を優先し、Average列で昇順にソート
df2 = df2.sort_values(["Unit","Average"], ascending=[True,False])

#DataFrameを切り分け、各Unitの最終行に合計を入れ、総合計も入れる
##控除
df2_Deduc = df2[df2["Unit"].isin(["1.Deduction"])]
df2_DeducAll = pd.concat([df2_Deduc,pd.DataFrame(df2_Deduc.sum(numeric_only=True)).T])
##固定費
df2_Fixed = df2[df2["Unit"].isin(["2.FixedExpenses"])]
df2_FixedAll = pd.concat([df2_Fixed,pd.DataFrame(df2_Fixed.sum(numeric_only=True)).T])
##変動費
df2_Variable = df2[df2["Unit"].isin(["3.VariableExpenses"])]
df2_VariableAll = pd.concat([df2_Variable,pd.DataFrame(df2_Variable.sum(numeric_only=True)).T])
##特別支出
df2_Extra = df2[df2["Unit"].isin(["4.ExtraordinaryExpenses"])]
df2_ExtraAll = pd.concat([df2_Extra,pd.DataFrame(df2_Extra.sum(numeric_only=True)).T])
##総合計のため上で作ったdf2をもってきて、合計する
df2_sum = df2.sum(numeric_only=True)
##合計を含んだ各Unitをつなぎ合わせる
df2_alldata = pd.concat([df2_DeducAll,df2_FixedAll,df2_VariableAll,df2_ExtraAll],axis=0)
###最後に総合計もつなぎ合わせ、すべてつなぎ合わせたら終わり！！！
df2_alldata = pd.concat([df2_alldata,pd.DataFrame(df2_sum).T])
print(df2_alldata)

df_expenditure = df2_alldata
#print(df_expenditure)

#スプレッドシートに書き出し
from gspread_dataframe import set_with_dataframe
expenditure ='[【ここにGoogleDriveに保存されているスプレッドシート（Expenditure_Sheet）のURLを挿入する】]'
worksheet = gc.open_by_url(expenditure).get_worksheet(0)
set_with_dataframe(worksheet, df_expenditure)




NoValidUrlKeyFound: 

# 2024_Profit&Lossコード
